In [ ]:
import pandas as pd
import numpy as np
import openpyxl
import xlrd
import importlib
import class_spark
import os
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import load_workbook
import excel


path = r'C:\\Users\\alexsandrosilva\\Downloads\\'

In [ ]:
#Convert excel date to Datetime
def read_date(date):
    try:
        return xlrd.xldate.xldate_as_datetime(date, 0)
    except Exception as err:
        raise Exception(err)

def update_xlsx(df, dest, sheetname, start_row, start_col,index=False, header=False):
    #convert dataframe to rows
    rows = dataframe_to_rows(df, index=index, header=header)
    #Open an xlsx for reading
    wb = load_workbook(filename = dest)
    #Get the current Active Sheet
    ws = wb[sheetname]
    #write
    for r_idx, row in enumerate(rows, start_row):
        for c_idx, value in enumerate(row, start_col):
             ws.cell(row=r_idx, column=c_idx).value = value
    #save the csb file
    wb.save(dest)
    wb.close()

def multiple_form(dest, sheetname, base_cell, n_lines):
    #Open an xlsx for reading
    wb = load_workbook(filename = dest)
    #Get the current Active Sheet
    ws = wb[sheetname]
    ex = excel.Excel()
    ex.copy_and_paste_formula(ws, base_cell, n_lines)
    wb.save(dest)
    wb.close()

In [ ]:
import re
from typing import List
from string import ascii_uppercase
import pythoncom
import win32com.client
from openpyxl.formula.translate import Translator


class DataWrite:
    def __init__(self, data: List[str], init_cell: str):
        self.__data: List[str] = data
        self.__init_cell: str = init_cell

    @property
    def data(self) -> List[str]:
        return self.__data

    @property
    def init_cell(self) -> str:
        return self.__init_cell

class Excel:
    def __init__(self):
        pass

    @staticmethod
    def insert_data(ws, cell: str, data: str):
        line = int(re.findall("[0-9].*", cell)[0])
        column = re.findall("[A-Z]{1,3}", cell)[0]
        columns_list = Excel.get_columns_names_by_range("A", column)
        column_number = len(columns_list)
        ws.cell(row=line, column=column_number).value = data

    @staticmethod
    def copy_and_paste_formula(ws, base_cell: str, n_lines: int):
        line = int(re.findall("[0-9].*", base_cell)[0])
        column = re.findall("[A-Z]{1,3}", base_cell)[0]
        for n in range(line + 1, n_lines + line):
            write_cell = f"{column}{str(n)}"
            ws[write_cell] = Translator(ws[base_cell].value, origin=base_cell).translate_formula(write_cell)

    @staticmethod
    def insert_mutiple_data(ws, datas_write: List[DataWrite]):
        for data_write in datas_write:
            line = re.findall("[0-9].*", data_write.init_cell)[0]
            column = re.findall("[A-Z]{1,3}", data_write.init_cell)[0]
            for step, data in enumerate(data_write.data):
                Excel.insert_data(ws=ws, cell=f"{column}{str(int(line) + step)}", data=data)

    @staticmethod
    def copy_and_paste_mutiple_formulas(ws, reference_cells: List[str], n_lines):
        for cell in reference_cells:
            Excel.copy_and_paste_formula(ws=ws, base_cell=cell, n_lines=n_lines)

    @staticmethod
    def get_columns_names_by_range(init: str, end: str) -> List[str]:
        l = list(ascii_uppercase)
        b = l + [x + y for x in l for y in l]
        b = b[b.index(init):b.index(end) + 1]

        return b

    @staticmethod
    def refresh_wb(path: str):
        pythoncom.CoInitialize()
        xlapp = win32com.client.DispatchEx('Excel.Application')
        excel = xlapp.Workbooks.Open(path)
        excel.RefreshAll()
        excel.Save()
        xlapp.Quit()